# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 04**: Data Unions & Joins Pipeline

**Date**: September 23rd 2025

**Student Name**: Jaime Enrique Galindo Villegas

**Professor**: Pablo Camarillo Ramirez

In [33]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on SparkSQL") \
    .master("spark://9d7a5c0179a1:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

In [34]:
!ls /opt/spark/work-dir/data/car_service/

agencies  brands  cars	customers  rentals


In [35]:
# Build schema
# Import your module
from jaime_galindo.spark_utils import SparkUtils
from pyspark.sql.functions import get_json_object


In [36]:

# Generar esquema
schema_agencies = SparkUtils.generate_schema([("agency_id", "int"), ("agency_info", "string")])

# Generar df
df_agencies = spark.read.schema(schema_agencies).option("header", True).csv("/opt/spark/work-dir/data/car_service/agencies")

# Crear las nuevas columnas
df_agencies = df_agencies.withColumn("agency_name", get_json_object(df_agencies.agency_info, "$.agency_name"))
df_agencies = df_agencies.withColumn("city", get_json_object(df_agencies.agency_info, "$.city"))

df_agencies.show(truncate=False)


+---------+-----------------------------------------------------+-------------+-------------+
|agency_id|agency_info                                          |agency_name  |city         |
+---------+-----------------------------------------------------+-------------+-------------+
|1        |{'agency_name': 'NYC Rentals', 'city': 'New York'}   |NYC Rentals  |New York     |
|2        |{'agency_name': 'LA Car Rental', 'city': 'Londres'}  |LA Car Rental|Londres      |
|3        |{'agency_name': 'Zapopan Auto', 'city': 'Zapopan'}   |Zapopan Auto |Zapopan      |
|4        |{'agency_name': 'SF Cars', 'city': 'San Francisco'}  |SF Cars      |San Francisco|
|5        |{'agency_name': 'Mexico Cars', 'city': 'Mexico City'}|Mexico Cars  |Mexico City  |
+---------+-----------------------------------------------------+-------------+-------------+



In [37]:
# Generar esquema
schema_brands = SparkUtils.generate_schema([("brand_id", "int"), ("brand_info", "string")])

# Generar df
df_brands = spark.read.schema(schema_brands).option("header", True).csv("/opt/spark/work-dir/data/car_service/brands")

# Crear las nuevas columnas
df_brands = df_brands.withColumn("brand_name", get_json_object(df_brands.brand_info, "$.brand_name"))
df_brands = df_brands.withColumn("country", get_json_object(df_brands.brand_info, "$.country"))

df_brands.show(truncate=False)

+--------+-----------------------------------------------------+-------------+-------+
|brand_id|brand_info                                           |brand_name   |country|
+--------+-----------------------------------------------------+-------------+-------+
|1       |{'brand_name': 'Mercedes-Benz', 'country': 'Germany'}|Mercedes-Benz|Germany|
|2       |{'brand_name': 'BMW', 'country': 'Germany'}          |BMW          |Germany|
|3       |{'brand_name': 'Audi', 'country': 'Germany'}         |Audi         |Germany|
|4       |{'brand_name': 'Ford', 'country': 'US'}              |Ford         |US     |
|5       |{'brand_name': 'BYD', 'country': 'China'}            |BYD          |China  |
|6       |{'brand_name': 'Honda', 'country': 'Japan'}          |Honda        |Japan  |
|7       |{'brand_name': 'Toyota', 'country': 'Japan'}         |Toyota       |Japan  |
+--------+-----------------------------------------------------+-------------+-------+



In [38]:
# Generar esquema
schema_cars = SparkUtils.generate_schema([("car_id", "int"), ("car_info", "string")])

# Generar df
df_cars = spark.read.schema(schema_cars).option("header", True).csv("/opt/spark/work-dir/data/car_service/cars")

# Crear las nuevas columnas
df_cars = df_cars.withColumn("car_name", get_json_object(df_cars.car_info, "$.car_name"))
df_cars = df_cars.withColumn("brand_id", get_json_object(df_cars.car_info, "$.brand_id"))
df_cars = df_cars.withColumn("price_per_day", get_json_object(df_cars.car_info, "$.price_per_day"))

df_cars.show(truncate=False)

+------+----------------------------------------------------------------------------------------+------------------------------------+--------+-------------+
|car_id|car_info                                                                                |car_name                            |brand_id|price_per_day|
+------+----------------------------------------------------------------------------------------+------------------------------------+--------+-------------+
|1     |{'car_name': 'Chang-Fisher Model 7', 'brand_id': 5, 'price_per_day': 139}               |Chang-Fisher Model 7                |5       |139          |
|2     |{'car_name': 'Sheppard-Tucker Model 4', 'brand_id': 6, 'price_per_day': 70}             |Sheppard-Tucker Model 4             |6       |70           |
|3     |{'car_name': 'Faulkner-Howard Model 5', 'brand_id': 3, 'price_per_day': 53}             |Faulkner-Howard Model 5             |3       |53           |
|4     |{'car_name': 'Wagner LLC Model 1', 'brand_id

In [39]:
# Generar esquema
schema_customers = SparkUtils.generate_schema([("customer_id", "int"), ("customer_info", "string")])

# Generar df
df_customers = spark.read.schema(schema_customers).option("header", True).csv("/opt/spark/work-dir/data/car_service/customers/")

# Crear las nuevas columnas
df_customers = df_customers.withColumn("customer_name", get_json_object(df_customers.customer_info, "$.customer_name"))
df_customers = df_customers.withColumn("city", get_json_object(df_customers.customer_info, "$.city"))
df_customers = df_customers.withColumn("age", get_json_object(df_customers.customer_info, "$.age"))

df_customers.show(truncate=False)

+-----------+-------------------------------------------------------------------------+--------------------+-------------+---+
|customer_id|customer_info                                                            |customer_name       |city         |age|
+-----------+-------------------------------------------------------------------------+--------------------+-------------+---+
|1          |{'customer_name': 'Tiffany Riley', 'city': 'Monterrey', 'age': 32}       |Tiffany Riley       |Monterrey    |32 |
|2          |{'customer_name': 'Matthew Davies', 'city': 'Monterrey', 'age': 36}      |Matthew Davies      |Monterrey    |36 |
|3          |{'customer_name': 'Rebecca Miller', 'city': 'Mexico City', 'age': 30}    |Rebecca Miller      |Mexico City  |30 |
|4          |{'customer_name': 'Katelyn Mccoy', 'city': 'New York', 'age': 34}        |Katelyn Mccoy       |New York     |34 |
|5          |{'customer_name': 'Dana Dennis', 'city': 'Zapopan', 'age': 26}           |Dana Dennis         |Zap

In [40]:
# Generar esquema
schema_rentals = SparkUtils.generate_schema([("rental_id", "int"), ("rental_info", "string")])

# Generar df
df_rentals = spark.read.schema(schema_rentals).option("header", True).csv("/opt/spark/work-dir/data/car_service/rentals")

# Crear las nuevas columnas
df_rentals = df_rentals.withColumn("car_id", get_json_object(df_rentals.rental_info, "$.car_id"))
df_rentals = df_rentals.withColumn("customer_id", get_json_object(df_rentals.rental_info, "$.customer_id"))
df_rentals = df_rentals.withColumn("agency_id", get_json_object(df_rentals.rental_info, "$.agency_id"))

df_rentals.show(truncate=False)

+---------+--------------------------------------------------+------+-----------+---------+
|rental_id|rental_info                                       |car_id|customer_id|agency_id|
+---------+--------------------------------------------------+------+-----------+---------+
|11891    |{'car_id': 21, 'customer_id': 71, 'agency_id': 1} |21    |71         |1        |
|11892    |{'car_id': 11, 'customer_id': 52, 'agency_id': 2} |11    |52         |2        |
|11893    |{'car_id': 22, 'customer_id': 116, 'agency_id': 4}|22    |116        |4        |
|11894    |{'car_id': 5, 'customer_id': 107, 'agency_id': 1} |5     |107        |1        |
|11895    |{'car_id': 4, 'customer_id': 53, 'agency_id': 4}  |4     |53         |4        |
|11896    |{'car_id': 8, 'customer_id': 131, 'agency_id': 2} |8     |131        |2        |
|11897    |{'car_id': 23, 'customer_id': 66, 'agency_id': 3} |23    |66         |3        |
|11898    |{'car_id': 24, 'customer_id': 60, 'agency_id': 4} |24    |60         

In [41]:
sc.stop()
